# WD Classifiers

In [4]:
import numpy as np
import os
from scipy.io import loadmat
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn import preprocessing
from sklearn import metrics
from sklearn.utils import shuffle

## Reproduzindo os resultados de [3]

### Montando o Dataset

Segundo [1], cada autor possui 24 assinaturas genuínas e 30 forjas, mas aqui foi verificado que para alguns autores estão faltando algumas forjas (pelo menos nos features disponibilizados no github).

Como a quantidade de assinaturas faltando é pequena, a importância desse fato foi desconsiderada.

In [11]:
exploitation_real = list()
exploitation_forg = list()

sig_files = os.listdir('./features/gpds_signet/exploitation/')
sig_files.sort()

total_forg = 0
total_real = 0
incomplete_forg = 0
incomplete_real = 0

for i in range(300):
    sigs = loadmat('./features/gpds_signet/exploitation/' + sig_files[i])['features']
    exploitation_forg.append(sigs)
    incomplete_forg += 30 - sigs.shape[0]
    total_forg += sigs.shape[0]

for i in range(300, 600):
    sigs = loadmat('./features/gpds_signet/exploitation/' + sig_files[i])['features']
    exploitation_real.append(sigs)
    incomplete_real += 24 - sigs.shape[0]
    total_real += sigs.shape[0]
        
print 'missing forged samples: ' + str(incomplete_forg) + ' out of ' + str(total_forg)
print 'missing genuine samples: ' + str(incomplete_real) + ' out of ' + str(total_real)

missing forged samples: 95 out of 8905
missing genuine samples: 0 out of 7200


In [12]:
validation_real = list()
validation_forg = list()

sig_files = os.listdir('./features/gpds_signet/validation/')

sig_files.sort()

total_forg = 0
total_real = 0
incomplete_forg = 0
incomplete_real = 0

for i in range(50):
    sigs = loadmat('./features/gpds_signet/validation/' + sig_files[i])['features']
    validation_forg.append(sigs)
    incomplete_forg += 30 - sigs.shape[0]
    total_forg += sigs.shape[0]

for i in range(50, 100):
    sigs = loadmat('./features/gpds_signet/validation/' + sig_files[i])['features']
    validation_real.append(sigs)
    incomplete_real += 24 - sigs.shape[0]
    total_real += sigs.shape[0]
        
print 'missing forged samples: ' + str(incomplete_forg) + ' out of ' + str(total_forg)
print 'missing genuine samples: ' + str(incomplete_real) + ' out of ' + str(total_real)

missing forged samples: 0 out of 1500
missing genuine samples: 0 out of 1200


In [13]:
development_real = list()
development_forg = list()

sig_files = os.listdir('./features/gpds_signet/development/')
sig_files.sort()

total_forg = 0
total_real = 0
incomplete_forg = 0
incomplete_real = 0

for i in range(531):
    sigs = loadmat('./features/gpds_signet/development/' + sig_files[i])['features']
    development_forg.append(sigs)
    incomplete_forg += 30 - sigs.shape[0]
    total_forg += sigs.shape[0]

for i in range(531, 531 * 2):
    sigs = loadmat('./features/gpds_signet/development/' + sig_files[i])['features']
    development_real.append(sigs)
    incomplete_real += 24 - sigs.shape[0]
    total_real += sigs.shape[0]
        
print 'missing forged samples: ' + str(incomplete_forg) + ' out of ' + str(total_forg)
print 'missing genuine samples: ' + str(incomplete_real) + ' out of ' + str(total_real)

missing forged samples: 21 out of 15909
missing genuine samples: 0 out of 12744


Como sanity check é bom verificar se o Exploitation, Development e Validation possuem 300, 531 e 50 autores respectivavmente.

Além disso, cada elemento das listas deve conter o formato (x, 2048). Somente o primeiro será verificado.

In [14]:
assert len(exploitation_forg) == 300
assert len(exploitation_real) == 300
assert len(validation_real) == 50
assert len(validation_forg) == 50
assert len(development_forg) == 531
assert len(development_real) == 531

assert exploitation_forg[0].shape == (exploitation_forg[0].shape[0], 2048)
assert exploitation_real[0].shape == (exploitation_real[0].shape[0], 2048)
assert validation_real[0].shape == (validation_real[0].shape[0], 2048)
assert validation_forg[0].shape == (validation_forg[0].shape[0], 2048)
assert development_forg[0].shape == (development_forg[0].shape[0], 2048)
assert development_real[0].shape == (development_real[0].shape[0], 2048)

### Encontrando os hiperparâmetros do SVM

Em [1] são usados 10 autores de D seguindo o mesmo protocolo do Exploitation Set para procurar pelos hiperparâmetros do SVM. Em [2] não é mencionado como, nem se, os hiperparâmetros são buscados, acredito que eles sejam buscados no Validation set ou que os mesmos valores encontrados em [1] sejam usados.

Existe um problema com a abordagem de [1]. Como as amostras negativas contém assinaturas genuínas de todos os autores de D, elas contém assinaturas genuínas do autor para o qual está se fazendo a validacão como amostras negativas (o que é incorreto). Decidi portanto usado o Validadtion set para buscas os hiperparâmetros do SVM mesmo sem saber o procedimento usado em [2].

Tanto em [1] quanto em [2], A busca por hiperparâmetros é feita apenas com o kernel RBF.

Em [3] não é feita uma busca hiperparâmetros.

In [15]:
# The negative samples are genuine signatures of authors pron the development set
train_negatives = list()
for i in range(len(development_forg)):
    author = development_real[i]
    train_negatives += list(author[:14])
    
train_negatives = np.array(train_negatives)
print train_negatives.shape

(7434, 2048)


In [16]:
score_counter = np.array([
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
], dtype='float32')

for author_id in range(50):
    print 'author ' + str(author_id + 1) + ' out of 50'
    for gamma_index, gamma in enumerate([-9, -11, -13]):
        for c_index, C in enumerate([1, 3, 10]):
            print '%.2f %% \r' % ((c_index + gamma_index * 5)/19.0 * 100),
            
            x_train = np.concatenate((validation_real[author_id][:14], train_negatives))
            y_train = np.concatenate((
                np.zeros(validation_real[author_id][:14].shape[0]), 
                np.ones(train_negatives.shape[0])
            ))

            x_test = np.concatenate((validation_real[author_id][14:], validation_forg[author_id]))
            y_test = np.concatenate((
                np.zeros(validation_real[author_id][14:].shape[0]),
                np.ones(validation_forg[author_id].shape[0])
            ))

            scaler = preprocessing.StandardScaler().fit(x_train)
            scaler.transform(x_train)
            scaler.transform(x_test)

            clf = svm.SVC(
                kernel='rbf', 
                C=C,
                gamma=2**(gamma),
                # it is advisable to set probability=False and use decision_function instead of predict_proba
                probability=False,
                class_weight='balanced'
            )
            clf.fit(x_train, y_train)
            score_counter[gamma_index][c_index] += clf.score(x_test, y_test)

print score_counter


author 1 out of 50
author 2 out of 50
author 3 out of 50
author 4 out of 50
author 5 out of 50
author 6 out of 50
author 7 out of 50
author 8 out of 50
author 9 out of 50
author 10 out of 50
author 11 out of 50
author 12 out of 50
author 13 out of 50
author 14 out of 50
author 15 out of 50
author 16 out of 50
author 17 out of 50
author 18 out of 50
author 19 out of 50
author 20 out of 50
author 21 out of 50
author 22 out of 50
author 23 out of 50
author 24 out of 50
author 25 out of 50
author 26 out of 50
author 27 out of 50
author 28 out of 50
author 29 out of 50
author 30 out of 50
author 31 out of 50
author 32 out of 50
author 33 out of 50
author 34 out of 50
author 35 out of 50
author 36 out of 50
author 37 out of 50
author 38 out of 50
author 39 out of 50
author 40 out of 50
author 41 out of 50
author 42 out of 50
author 43 out of 50
author 44 out of 50
author 45 out of 50
author 46 out of 50
author 47 out of 50
author 48 out of 50
author 49 out of 50
author 50 out of 50
[[ 45.299

Em [1] encontra-se C = 1 e gamma = 2⁻¹² para o kernel RBF. Para o kernel linear usa-se C=1.

Em [2] encontra-se C = 1 egamma = 2⁻¹¹ para o kernel RBF. Para o kernel linear usa-se C=1.

Em [3] usa-se os mesmos valores de [2].

Aqui encontra-se C = 1 e gamma = 2⁻⁹. Para o kernel RBF. Não foi usado o kernel linear porque ele sempre tinha um desempenho inferior.

### Treinando e testando o modelo

Treino e teste são feitos individualmente para cada autor do Exploitation set.


##### Treino
Como amostras positivas serão usadas 12 das 24 assinaturas genuínas disponíveis para cada autor no Exploitation set. Como amostras negativas serão usadas 12 assinaturas de CADA autor do development set (isso é feito porque não faz sentido usar forjas durante o treino).

##### Teste
Como amostras positivas serão usadas as 12 assinaturas genuínas restantes e como negativas serão usadas todas as forjas disponíveis.

Como todos os autores usam as mesmas amostras negativas, esse conjunto será montado antes de percorrer os autores

In [17]:
eers = list()

for run in range(10):
    print 'run', (run + 1)
    sum_eer = 0
    sum_eer_size = 0

    for author_id in range(300):
        author_exploitation_real = shuffle(exploitation_real[author_id])
        author_exploitation_forg = shuffle(exploitation_forg[author_id])
        print str(author_id + 1) + ' out of 300\r',

        x_train = np.concatenate((author_exploitation_real[:12], train_negatives))
        y_train = np.concatenate((
            np.zeros(author_exploitation_real[:12].shape[0]),
            np.ones(train_negatives.shape[0])
        ))

        x_test = np.concatenate((author_exploitation_real[12:], author_exploitation_forg))
        y_test = np.concatenate((
            np.zeros(author_exploitation_real[12:].shape[0]),
            np.ones(author_exploitation_forg.shape[0])
        ))

        scaler = preprocessing.StandardScaler().fit(x_train)
        scaler.transform(x_train)
        scaler.transform(x_test)

        clf = svm.SVC(
            kernel='rbf', 
            C=1,
            gamma=2**(-9), # -9 or -11
            # it is advisable to set probability=False and use decision_function instead of predict_proba
            probability=False,
            class_weight='balanced'
        )
        clf.fit(x_train, y_train)

        y_scores = clf.decision_function(x_test)
        fpr, tpr, threshold = metrics.roc_curve(y_test, y_scores, pos_label=1)
        fnr = 1 - tpr
        eer_index = np.argmin(np.absolute(fnr - fpr))
        eer = (fpr[eer_index] + fnr[eer_index]) / 2
        sum_eer += eer
        sum_eer_size += 1
    print '                       \r',
    eers.append(sum_eer / sum_eer_size)
    print sum_eer / sum_eer_size

eers = np.array(eers)
print 'final results:'
print 'mean eer:', eers.mean()
print 'stddev:', eers.std()


run 1
0.032082688916         
run 2
0.0338927203065        
run 3
0.034681097638         
run 4
0.0302518853007        
run 5
0.0334306567209        
run 6
0.0355178869662        
run 7
0.0307833728638        
run 8


KeyboardInterrupt: 

### Resultados

##### Kernel linear
<strong>[3]</strong>: EER=3.91+-0.64 (média, desvio padrão). <br> 
<strong>Aqui</strong>: Não testado.

##### Kernel RBF
<strong>[3]</strong>: EER=3.13+-0.46 (média, desvio padrão). <br>
<strong>Aqui</strong>: EER=3.60+-0.15 com os parâmetros do artigo e EER=3.32+-0.17 com os encontrados aqui.

### Referências

[1] https://arxiv.org/pdf/1604.00974 <br>
[2] https://arxiv.org/pdf/1607.04573 <br>
[3] https://arxiv.org/pdf/1705.05787